In [57]:
import sys
sys.path.append('..')

In [58]:
import numpy as np
import pandas as pd
import pickle

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.optimizers import RMSprop, Adam, Adamax

from lib.read_data import read_and_join_output_file
#from lib.create_pipeline import create_transformation_pipeline
from lib.deeplearning import create_transformation_pipelines
from lib.transform_impute import convert_back_df
from lib.split_data import train_test_group_time_split

In [59]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [60]:
# During experiment we can try to use neptune.ai to log all the Tensorflow experiments results
neptune_key = pickle.load(open("./neptune.pkl", "rb"))


## Preparing the Dataset
The train and test sets are split by Township-Ranges, i.e. some Township-Ranges data are either fully in the train or test set.
The target value is the value of that variable for 2021
Thus train/test sets are of shape (number of Township-Ranges, 7 years (2014-2020), the number of features).
The input of 1 data point in the model is of shape (7x81


In [61]:
RANDOM_SEED = 42
# Load the data from the ETL output files
X = read_and_join_output_file()
#X["WELL_COUNT"] = X["WELL_COUNT_PUBLIC"] + X["WELL_COUNT_AGRICULTURE"] + X["WELL_COUNT_DOMESTIC"] + X["WELL_COUNT_INDUSTRIAL"]
#X.drop(columns=["WELL_COUNT_PUBLIC", "WELL_COUNT_AGRICULTURE", "WELL_COUNT_DOMESTIC", "WELL_COUNT_INDUSTRIAL"], inplace=True)
# Split the data into a training and a test set
X_train_df, X_test_df, y_train_df, y_test_df = train_test_group_time_split(X, index=["TOWNSHIP_RANGE", "YEAR"], group="TOWNSHIP_RANGE", random_seed=RANDOM_SEED)
# Create, fit and apply the data imputation pipeline to the training and test sets
impute_pipeline, columns = create_transformation_pipelines(X_train_df)
X_train_impute = impute_pipeline.fit_transform(X_train_df)
X_test_impute = impute_pipeline.transform(X_test_df)
# Convert the X_train and X_test back to dataframes
X_train_impute_df = pd.DataFrame(X_train_impute, index=X_train_df.index, columns=columns)
X_test_impute_df = pd.DataFrame(X_test_impute, index=X_test_df.index, columns=columns)
# Keep only the GSE_GWE variable as the outcome variable
scaler = MinMaxScaler()
y_train = scaler.fit_transform(y_train_df[["GSE_GWE"]])
y_test = scaler.transform(y_test_df[["GSE_GWE"]])
X_train_impute_df

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R03E      2014             0.097778       0.018246              0.037145   
               2015             0.095238       0.021053              0.025042   
               2016             0.114286       0.007916              0.022398   
               2017             0.000000       0.013684              0.030885   
               2018             0.083873       0.002474              0.034558   
...                                  ...            ...                   ...   
T32S R30E      2016             0.000000       0.000000              0.000000   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R03E      2014                     0.098039   
               2015                     0.117647   
               2016                     0.152614   
               2017                     0.127451   
               2018                     0.148257   
...                                          ...   
T32S R30E      2016                     0.000000   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R03E      2014                        0.111111                 0.105856   
               2015                        0.080460                 0.079848   
               2016                        0.103768                 0.104880   
               2017                        0.082375                 0.081749   
               2018                        0.093934                 0.107605   
...                                             ...                      ...   
T32S R30E      2016                        0.000000                 0.000000   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R03E      2014                       0.000037   
               2015                       0.000037   
               2016                       0.000037   
               2017                       0.000037   
               2018                       0.000037   
...                                            ...   
T32S R30E      2016                       0.033178   
               2017                       0.033178   
               2018                       0.033178   
               2019                       0.033178   
               2020                       0.033178   

                     VEGETATION_CALIFORNIA_COAST_LIVE_OAK  \
TOWNSHIP_RANGE YEAR                                         
T01N R03E      2014                              0.000137   
               2015                              0.000137   
               2016                              0.000137   
               2017                              0.000137   
               2018                              0.000137   
...                                                   ...   
T32S R30E      2016                              0.000000 

In [62]:
# Change the shape of the input array to (number of Township-Ranges, 7 years (2014-2020), the number of features)
X_train = X_train_impute_df.values.reshape(len(X_train_impute_df.index.get_level_values(0).unique()), len(X_train_impute_df.index.get_level_values(1).unique()), X_train_impute_df.shape[1])
X_test = X_test_impute_df.values.reshape(len(X_test_impute_df.index.get_level_values(0).unique()), len(X_test_impute_df.index.get_level_values(1).unique()), X_test_impute_df.shape[1])

In [63]:
print("="*100)
print("Checking the train, validation and test input (X) datasets sizes:")
print(f"Size of the X_train dataset: {X_train.shape}")
#print(f"Size of the X_val dataset: {X_val.shape}")
print(f"Size of the X_test dataset: {X_test.shape}")
print("="*100)
print("Checking the train, validation and test output (y) datasets sizes:")
print(f"Size of the y_train dataset: {y_train.shape}")
#print(f"Size of the y_val dataset: {y_val_df.shape}")
print(f"Size of the y_test dataset: {y_test.shape}")

Checking the train, validation and test input (X) datasets sizes:
Size of the X_train dataset: (382, 7, 81)
Size of the X_test dataset: (96, 7, 81)
Checking the train, validation and test output (y) datasets sizes:
Size of the y_train dataset: (382, 1)
Size of the y_test dataset: (96, 1)


In [64]:
results_df = y_test_df[["GSE_GWE"]].copy()

In [76]:
nb_features = len(X_train_impute_df.columns)

hyper_parameters = {
    "validation_split": 0.05,
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 30,
    "lstm_units": 200,
    "lstm_activation": "tanh",
    "output_activation": "linear",
    "nb_features": nb_features,
    "optimizer": "RMSprop"
}

optimizer = {
    "RMSprop": RMSprop(learning_rate=hyper_parameters["learning_rate"]),
    "Adam": Adam(learning_rate=hyper_parameters["learning_rate"]),
    "Adamax": Adamax(learning_rate=hyper_parameters["learning_rate"])
}

In [77]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
    mse_ = keras.metrics.MeanSquaredError()
    mae_ = keras.metrics.MeanAbsoluteError()
    rmse_ = keras.metrics.RootMeanSquaredError()
    mae = mae_(y_test_inverse,yhat_inverse)
    print('mae:', mae)
    mse = mse_(y_test_inverse,yhat_inverse)
    print('mse:', mse)
    rmse = rmse_(y_test_inverse,yhat_inverse)
    print('rmse:', rmse)
    return mae, mse, rmse

In [78]:
model1 = Sequential()
model1.add(LSTM(hyper_parameters["lstm_units"], activation=hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation=hyper_parameters["output_activation"]))
model1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 200)               225600    
                                                                 
 dense_12 (Dense)            (None, 1)                 201       
                                                                 
Total params: 225,801
Trainable params: 225,801
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Start experiment
run = neptune.init(
    project="milestone2-california-water-shortage/deeplearning-lstm",
    api_token=neptune_key,
    name="Basic Model",
    tags=["WithDetailedWellCounts", "UnidirectionalLSTM"]
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
run['hyper-parameters'] = hyper_parameters

model1.compile(loss="mse", optimizer=optimizer[hyper_parameters["optimizer"]])
history = model1.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_1_prediction"] = yhat_inverse
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/milestone2-california-water-shortage/deeplearning-lstm/e/DEEPLSTM-50
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/30
12/12 [==============================] - 3s 133ms/step - loss: 0.0301 - val_loss: 0.0097
Epoch 2/30
12/12 [==============================] - 1s 102ms/step - loss: 0.0092 - val_loss: 0.0066
Epoch 3/30
12/12 [==============================] - 1s 73ms/step - loss: 0.0077 - val_loss: 0.0134
Epoch 4/30
12/12 [==============================] - 1s 60ms/step - loss: 0.0056 - val_loss: 0.0093
Epoch 5/30
12/12 [==============================] - 1s 57ms/step - loss: 0.0049 - val_loss: 0.0198
Epoch 6/30
12/12 [==============================] - 1s 61ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 7/30
12/12 [==============================] - 1s 58ms/step - loss: 0.0041 - val_loss: 0.

In [82]:
hyper_parameters["dense_units"] = 30
hyper_parameters["dense_activation"] = "tanh"
hyper_parameters["dropout"] = 0.02

model2 = Sequential()
model2.add(Bidirectional(LSTM(hyper_parameters["lstm_units"], activation=hyper_parameters["lstm_activation"]), input_shape=(7, nb_features)))
model2.add(Dense(hyper_parameters["dense_units"], activation=hyper_parameters["dense_activation"]))
model2.add(Dropout(hyper_parameters["dropout"]))
model2.add(Dense(1, activation=hyper_parameters["output_activation"]))
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 400)              451200    
 nal)                                                            
                                                                 
 dense_13 (Dense)            (None, 30)                12030     
                                                                 
 dropout_4 (Dropout)         (None, 30)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 463,261
Trainable params: 463,261
Non-trainable params: 0
_________________________________________________________________


In [83]:
run = neptune.init(
    project="milestone2-california-water-shortage/deeplearning-lstm",
    api_token=neptune_key,
    name="Advanced Model 1",
    tags=["WithDetailedWellCounts", "BidirectionalLSTM"]
)
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')
run['hyper-parameters'] = hyper_parameters

model2.compile(loss="mse", optimizer=optimizer[hyper_parameters["optimizer"]])
history = model2.fit(X_train, y_train,
                     validation_split=hyper_parameters["validation_split"],
                     batch_size=hyper_parameters["batch_size"],
                     epochs=hyper_parameters["epochs"],
                     shuffle=True,
                     callbacks=[neptune_cbk])
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = scaler.inverse_transform(yhat)
y_test_inverse = scaler.inverse_transform(y_test)
results_df["experiment_2_prediction"] = yhat_inverse
evaluate_forecast(y_test_inverse, yhat_inverse)
mae, mse, rmse = evaluate_forecast(y_test_inverse, yhat_inverse)
run["eval/mae"] = mae
run["eval/mse"] = mse
run["eval/rmse"] = rmse
run.stop()

https://app.neptune.ai/milestone2-california-water-shortage/deeplearning-lstm/e/DEEPLSTM-51
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/30
12/12 [==============================] - 6s 272ms/step - loss: 0.0746 - val_loss: 0.0081
Epoch 2/30
12/12 [==============================] - 2s 151ms/step - loss: 0.0089 - val_loss: 0.0044
Epoch 3/30
12/12 [==============================] - 2s 153ms/step - loss: 0.0114 - val_loss: 0.0089
Epoch 4/30
12/12 [==============================] - 2s 164ms/step - loss: 0.0111 - val_loss: 0.0079
Epoch 5/30
12/12 [==============================] - 3s 273ms/step - loss: 0.0121 - val_loss: 0.0059
Epoch 6/30
12/12 [==============================] - 2s 188ms/step - loss: 0.0060 - val_loss: 0.0083
Epoch 7/30
12/12 [==============================] - 2s 153ms/step - loss: 0.0081 - val_los

In [84]:
results_df

,,GSE_GWE,experiment_1_prediction,experiment_2_prediction,experiment_3_prediction,experiment_4_prediction
TOWNSHIP_RANGE,YEAR,,,,,
T01N R02E,2021,53.193636,36.336391,121.115761,58.880062,55.766125
T01N R11E,2021,107.955000,142.946442,191.187546,145.260483,132.496689
T01S R03E,2021,24.494538,18.849447,89.398743,34.111828,28.095566
T01S R07E,2021,38.644000,23.731907,116.964813,56.066540,45.322628
T01S R10E,2021,113.651250,104.674149,146.275101,103.688248,92.169777
...,...,...,...,...,...,...
T31S R26E,2021,173.915909,179.801483,218.060379,229.395309,228.809418
T31S R31E,2021,403.900000,368.786682,469.795868,381.163025,324.155060
T32S R22E,2021,160.340000,200.964478,257.061676,194.516525,186.001938


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b042e2da-6536-449d-95b8-d85fa08825de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>